<a href="https://colab.research.google.com/github/GoutamChaitanya72/WSB/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Stock Analysis and Technical Indicators
This notebook fetches stock data for user-specified tickers and calculates various technical indicators, including moving averages, Fibonacci retracement levels, and support/resistance levels. It also provides simple buy/sell recommendations based on the calculated indicators.

In [1]:
!pip install pandas_ta ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 944.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=3b60c8ffeed04c56987ce61b220f1cfca37f25ee5490342b824016ed5f6719a6
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
# Import required libraries
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Fetch stock data
def fetch_stock_data(ticker, period='1y', interval='1d'):
    stock = yf.Ticker(ticker)
    data = stock.history(period=period, interval=interval)
    return data

In [ ]:
# Add technical indicators
def add_technical_indicators(data):
    data['SMA_20'] = ta.sma(data['Close'], length=20)
    data['SMA_50'] = ta.sma(data['Close'], length=50)
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['MACD'], data['MACD_Signal'], _ = ta.macd(data['Close'])
    return data

In [ ]:
# Calculate moving averages
def calculate_moving_averages(data):
    return {
        "20-Day MA": data['Close'].rolling(window=20).mean().iloc[-1],
        "50-Day MA": data['Close'].rolling(window=50).mean().iloc[-1],
        "200-Day MA": data['Close'].rolling(window=200).mean().iloc[-1]
    }

In [ ]:
# Calculate Fibonacci retracement levels
def calculate_fibonacci_levels(data):
    high_range = data['High'].max()
    low_range = data['Low'].min()
    difference = high_range - low_range
    return {
        "0.0% (Swing High)": high_range,
        "23.6%": high_range - 0.236 * difference,
        "38.2%": high_range - 0.382 * difference,
        "50.0%": high_range - 0.5 * difference,
        "61.8%": high_range - 0.618 * difference,
        "100.0% (Swing Low)": low_range
    }

In [ ]:
# Calculate support and resistance levels
def calculate_support_resistance(data):
    high_range = data['High'].max()
    low_range = data['Low'].min()

    # Initialize levels
    levels = {"S1": None, "S2": None, "S3": None, "R1": None, "R2": None, "R3": None}

    # Calculate support and resistance
    levels["R1"] = high_range
    levels["S1"] = low_range

    difference = high_range - low_range
    levels["R2"] = high_range - 0.382 * difference
    levels["R3"] = high_range - 0.618 * difference
    levels["S2"] = low_range + 0.382 * difference
    levels["S3"] = low_range + 0.618 * difference

    return levels

In [ ]:
# Simple buy/sell recommendation based on SMA and RSI
def simple_recommendation(data):
    last_rsi = data['RSI'].iloc[-1]
    last_price = data['Close'].iloc[-1]
    sma_20 = data['SMA_20'].iloc[-1]
    sma_50 = data['SMA_50'].iloc[-1]

    if last_price > sma_20 and last_price > sma_50 and last_rsi < 70:
        return "Buy"
    elif last_price < sma_50 and last_rsi > 70:
        return "Sell"
    else:
        return "Hold"

In [ ]:
# Display results
def display_results(data, ticker, moving_averages, fib_levels, support_resistance, real_time_price):
    print(f"\nResults for {ticker}:")
    print(f"Close Price: {data['Close'].iloc[-1]:.2f}")

    print("\nMoving Averages:")
    for ma, value in moving_averages.items():
        print(f"{ma}: {value:.2f}")

    print("\nFibonacci Retracement Levels:")
    for level, value in fib_levels.items():
        print(f"{level}: {value:.2f}")

    print(f"\nReal-Time Price: {real_time_price:.2f}")

    print("\nSupport Levels:")
    print(f"S1: {support_resistance['S1']:.2f}")
    print(f"S2: {support_resistance['S2']:.2f}")
    print(f"S3: {support_resistance['S3']:.2f}")

    print("\nResistance Levels:")
    print(f"R1: {support_resistance['R1']:.2f}")
    print(f"R2: {support_resistance['R2']:.2f}")
    print(f"R3: {support_resistance['R3']:.2f}")

    # Get buy/sell recommendation
    recommendation = simple_recommendation(data)
    print(f"Recommendation: {recommendation}")

In [ ]:
# Run analysis and display results
def run_analysis(tickers):
    clear_output()  # Clear previous outputs
    for ticker in tickers:
        print(f"\nRunning analysis for {ticker}...")

        # Fetch data
        data = fetch_stock_data(ticker)

        # Check if data is available
        if data.empty:
            print(f"No data available for {ticker}. Please check the ticker symbol.")
            continue

        # Add technical indicators
        data = add_technical_indicators(data)

        # Calculate moving averages
        moving_averages = calculate_moving_averages(data)

        # Calculate Fibonacci retracement levels
        fib_levels = calculate_fibonacci_levels(data)

        # Calculate support and resistance levels
        support_resistance = calculate_support_resistance(data)

        # Fetch real-time price
        real_time_price = yf.Ticker(ticker).history(period="1d")['Close'].iloc[0]

        # Display results
        display_results(data, ticker, moving_averages, fib_levels, support_resistance, real_time_price)

In [3]:
# Input tickers dynamically using a widget
ticker_input = widgets.Text(
    description='Tickers:',
    placeholder='AAPL, MSFT'
)

run_button = widgets.Button(description='Run Analysis')
output_area = widgets.Output()

def on_run_button_clicked(b):
    tickers = ticker_input.value.split(',')
    run_analysis([ticker.strip().upper() for ticker in tickers])

run_button.on_click(on_run_button_clicked)

# Display input and button
display(ticker_input, run_button, output_area)

Text(value='', description='Tickers:', placeholder='AAPL, MSFT')

Button(description='Run Analysis', style=ButtonStyle())

Output()